In [15]:
import pandas as pd
import numpy as np
import json, os, time
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
 
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'basics_filtered.csv',
 'IMDB Movie Dataset Info.docx',
 'ratings_filtered.csv',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']

In [16]:
with open('/Users/austi/.secret/tmdb_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [17]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

In [18]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    
    movie_info = movie.info()
    releases = movie.releases()
    
    for c in releases['countries']:
        if c['iso_3166_1' ] =='US':
            movie_info['certification'] = c['certification']
    return movie_info


def write_json(new_data, filename): 
    with open(filename,'r+') as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file)

In [19]:
basics = pd.read_csv('Data/basics_filtered.csv')
basics = basics.drop(columns='Unnamed: 0')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
86974,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
86975,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
86976,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
86977,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [24]:
YEAR = [2001,2002]
errors =[]

In [28]:
for y in YEAR:
    JSON_FILE = f'{FOLDER}tmdb_api_results_{y}.json'

    file_exists = os.path.isfile(JSON_FILE)

    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={y}.")

        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    else:
        print(f'The file {JSON_FILE} already exists.')

    df = basics.loc[basics['startYear']==y].copy()
    movie_ids = df['tconst']
    movie_ids.head()

    previous_df = pd.read_json(JSON_FILE)
    previous_df

    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {y}"):

        try:
            temp = get_movie_with_rating(movie_id)
            write_json(temp,JSON_FILE)
            time.sleep(0.02)
        except Exception as e:
            print(f'[!] Error {movie_id}')
            errors.append([movie_id, e])
    final_year_df = pd.read_json(JSON_FILE)

    csv_fname = f"{FOLDER}final_tmdb_data_{y}.csv.gz"
    final_year_df.to_csv(csv_fname, compression="gzip", index=False)

Creating Data/tmdb_api_results_2001.json for API results for year=2001.


Movies from 2001:   0%|          | 0/1576 [00:00<?, ?it/s]

[!] Error tt0151685
[!] Error tt0160270
[!] Error tt0160480
[!] Error tt0192110
[!] Error tt0193365
[!] Error tt0193537
[!] Error tt0195108
[!] Error tt0198748
[!] Error tt0200525
[!] Error tt0203206
[!] Error tt0206211
[!] Error tt0211279
[!] Error tt0215719
[!] Error tt0216290
[!] Error tt0217664
[!] Error tt0237199
[!] Error tt0246076
[!] Error tt0246409
[!] Error tt0247389
[!] Error tt0248017
[!] Error tt0254448
[!] Error tt0255348
[!] Error tt0256766
[!] Error tt0257947
[!] Error tt0258512
[!] Error tt0265521
[!] Error tt0266339
[!] Error tt0270179
[!] Error tt0271547
[!] Error tt0272543
[!] Error tt0273568
[!] Error tt0274491
[!] Error tt0274708
[!] Error tt0276473
[!] Error tt0276760
[!] Error tt0278839
[!] Error tt0279088
[!] Error tt0280682
[!] Error tt0280883
[!] Error tt0281015
[!] Error tt0282010
[!] Error tt0282220
[!] Error tt0283303
[!] Error tt0283572
[!] Error tt0284342
[!] Error tt0284357
[!] Error tt0284673
[!] Error tt0286713
[!] Error tt0286939
[!] Error tt0287123


Movies from 2002:   0%|          | 0/1572 [00:00<?, ?it/s]

[!] Error tt0128216
[!] Error tt0139297
[!] Error tt0152772
[!] Error tt0154187
[!] Error tt0181430
[!] Error tt0192789
[!] Error tt0193598
[!] Error tt0206778
[!] Error tt0220607
[!] Error tt0229567
[!] Error tt0251163
[!] Error tt0253016
[!] Error tt0253933
[!] Error tt0268992
[!] Error tt0281209
[!] Error tt0281231
[!] Error tt0284323
[!] Error tt0284664
[!] Error tt0287335
[!] Error tt0295456
[!] Error tt0296889
[!] Error tt0296933
[!] Error tt0298516
[!] Error tt0299689
[!] Error tt0300196
[!] Error tt0302962
[!] Error tt0303144
[!] Error tt0303327
[!] Error tt0303450
[!] Error tt0304008
[!] Error tt0304321
[!] Error tt0304478
[!] Error tt0306230
[!] Error tt0306967
[!] Error tt0307451
[!] Error tt0307618
[!] Error tt0308204
[!] Error tt0308244
[!] Error tt0308332
[!] Error tt0309474
[!] Error tt0309504
[!] Error tt0309803
[!] Error tt0309813
[!] Error tt0310315
[!] Error tt0310953
[!] Error tt0311328
[!] Error tt0311850
[!] Error tt0312011
[!] Error tt0312286
[!] Error tt0312662


In [29]:
print(f"- Total errors: {len(errors)}")

- Total errors: 529
